In [1]:
import os
import numpy as np
import pandas as pd

In [56]:
def remove_first_frames(df: pd.DataFrame):
    LANDMARKS = [f'x_{i}' for i in range(68)] + [f'y_{i}' for i in range(68)]
    firstFrame = 1
    for row in df[LANDMARKS].itertuples(index=True, name=None):
        if not any(row[1:]): 
            df.drop(row[0], inplace=True)
            firstFrame = int(df.iloc[0][0])
        else: break
    return df, firstFrame

In [131]:
def remove_last_frames(df: pd.DataFrame):
    LANDMARKS = [f'x_{i}' for i in range(68)] + [f'y_{i}' for i in range(68)]
    lastFrame = df.iloc[-1][0]
    
    lastValues = np.diff(df.x_0.values[-30:])>30
    if any(lastValues):
        lastValues = lastValues.tolist()
        lastFrame = -(len(lastValues)-lastValues.index(True))
        df = df.iloc[:lastFrame]
    
    return df, lastFrame

In [ ]:
broken_csvs = set()
base_dir = 'dataset/ElderReact_Data/ElderReact_train/'
for video in os.listdir(base_dir):
    video = 'battlefield1_50.mp4'
    videoName = video[:-4]
    openface_df = pd.read_csv(os.path.join('openFace/processed', f'{videoName}_openface.csv'))
    mediapipe_df = pd.read_csv(os.path.join('mediaPipe/processed', f'{videoName}_mediapipe.csv'))
    
    openface_df, openface_firstFrame = remove_first_frames(openface_df)
    mediapipe_df, mediapipe_firstFrame = remove_first_frames(mediapipe_df)
    
    if openface_firstFrame!=1:
        mediapipe_df = mediapipe_df.iloc[openface_firstFrame-1:, :]
    elif mediapipe_firstFrame!=1:
        openface_df = openface_df.iloc[mediapipe_firstFrame-1:, :]
    
    
    openface_df, openface_lastFrame = remove_last_frames(openface_df)
    mediapipe_df, mediapipe_lastFrame = remove_last_frames(mediapipe_df)
    
    if openface_lastFrame!=openface_df.iloc[-1][0]:
        print('open')
        mediapipe_df = mediapipe_df.iloc[:openface_lastFrame, :]
    elif mediapipe_lastFrame!=mediapipe_df.iloc[-1][0]:
        openface_df = openface_df.iloc[:mediapipe_lastFrame, :]
    
    openface_df.to_csv(f'openFace/processed_cleansed/{videoName}_openface_cleansed.csv')
    mediapipe_df.to_csv(f'mediaPipe/processed_cleansed/{videoName}_mediapipe_cleansed.csv')